In [1]:
import os

train_data_path = 'data/train/2025-04-11'
skip_files = ['stopwords']

def get_files(path: str) -> list:
    entries = os.listdir(path)
    return [os.path.join(path, entry) for entry in entries if os.path.isfile(os.path.join(path, entry))]

train_file_paths = get_files(train_data_path)
print(train_file_paths)

['data/train/2025-04-11\\1-相亲闪婚后，老公竟是千亿财阀.txt', 'data/train/2025-04-11\\10-腹黑娇妻：厉少，人设崩坏中.txt', 'data/train/2025-04-11\\100 小夫人一身反骨，大叔乖乖臣服.txt', 'data/train/2025-04-11\\101 墨爷的闪婚小甜妻.txt', 'data/train/2025-04-11\\102 流年不负你情深.txt', 'data/train/2025-04-11\\13-帝少甜婚令：娇妻，乖一点.txt', 'data/train/2025-04-11\\14-闪婚厚爱：病娇老公竟是千亿大佬.txt', 'data/train/2025-04-11\\17-闪婚三年不见面，再遇我是他秘书.txt', 'data/train/2025-04-11\\一夜惊喜：超级总裁喜当爹.txt', 'data/train/2025-04-11\\一胎六宝：总裁爸比超凶猛.txt', 'data/train/2025-04-11\\一胎双宝：季总你出局了.txt', 'data/train/2025-04-11\\一顾很倾情(www.sxcnw.org).txt', 'data/train/2025-04-11\\七年之痒，前夫不知道我重生了.txt', 'data/train/2025-04-11\\七爷的罪妻.txt', 'data/train/2025-04-11\\三宝联萌 大佬妈咪拽又飒.txt', 'data/train/2025-04-11\\三宝联萌：大佬妈咪拽又飒.txt', 'data/train/2025-04-11\\三年恋情曝光，我成了白月光的替身.txt', 'data/train/2025-04-11\\上善若玉(www.sxcnw.org).txt', 'data/train/2025-04-11\\不受宠？无所谓，杀疯豪门万人跪 (1~20章)-总裁、多重身份、傲慢霸气、复仇、男-有魅力的角色、浪漫、甜蜜、言情、大女主、甜宠.txt', 'data/train/2025-04-11\\不当舔狗后，肥妻她美又飒！ (1~20章)-多重身份、离婚、豪门霸总、真假千金、追妻火葬场、破镜重圆、甜宠、打脸.txt', 'd

In [2]:
from src.util.read_file import read_file

encoding = 'utf-8'
train_novels = list[tuple]()

for file_path in train_file_paths:
    try:
        train_novels.append((file_path, read_file(file_path, encoding)))
    except Exception:
        print('Decode Error ', file_path)

with open('data/stopwords', encoding='utf-8') as _f:
    stopwords = _f.readlines()
    stopwords = [x.replace('\n', '') for x in stopwords]
    
if len(train_novels) > 0:
    print(str(train_novels[0])[:512], '...')
print(stopwords[:128])

Decode Error  data/train/2025-04-11\一顾很倾情(www.sxcnw.org).txt
Decode Error  data/train/2025-04-11\七爷的罪妻.txt
Decode Error  data/train/2025-04-11\三宝联萌 大佬妈咪拽又飒.txt
Decode Error  data/train/2025-04-11\上善若玉(www.sxcnw.org).txt
Decode Error  data/train/2025-04-11\冷情总裁的退婚新娘.txt
Decode Error  data/train/2025-04-11\大佬你的小娇娇甜翻了.txt
Decode Error  data/train/2025-04-11\天才狂少的男妻.txt
Decode Error  data/train/2025-04-11\星河灿烂皆因你.txt
Decode Error  data/train/2025-04-11\暖婚急诊：商医生，来盖个章！.txt
('data/train/2025-04-11\\1-相亲闪婚后，老公竟是千亿财阀.txt', '\n第1章 来闪婚还摆架子\n书名： 相亲闪婚后，老公竟是千亿财阀 作者： 一板奶片 本章字数： 2164 更新时间： 2023-01-18 21:15:55\n\n“清溪，今天晚上你在外面吃个饭再回来了，没钱舅舅给你，你舅妈在发疯呢……”\n\n电话那头舅舅的话还没说完，又紧跟着传来舅妈尖锐的声音。\n\n“我发疯？你那贱种侄女今年26岁还在读书，不嫁人在我家吃我的用我的，还说我发疯？！现在公司遇到这么大的危机，我不管，你必须把她介绍给……”\n\n余清溪刚准备开口，电话匆忙地挂断了。\n\n她无声地叹了口气，看着自己刚发的奖学金，一份没动给舅舅转了过去。\n\n这样的情况，基本上每个星期都会出现。\n\n自从她妈妈去世后，她被舅舅接回家。因为舅妈不喜欢她，她只能在这个家小心翼翼地生活着，直到考上了文物修复学的研究生，她才将前些年欠舅舅的学费用奖学金还干净了。\n\n本想着，这个家无论如何她都得搬走了，苦于舅舅一直不同意，舅妈也因此一直跟他吵架。\n\n她唯一能想到能够正大光明搬走，舅舅不会担心的借口，就是结婚 ...
['!', '"', 

In [3]:
from concurrent.futures import ThreadPoolExecutor
from src.util.preprocess import split_words

with ThreadPoolExecutor(max_workers=8) as executor:
    train_seg_lists = list(executor.map(lambda _novel: (_novel[0], split_words(_novel[1], stopwords)), train_novels))
    
print(train_seg_lists[0])

Split: 8
Split: 7
Split: 9
Split: 8
Split: 10
Split: 7
Split: 7
Split: 7
These arent names: ['无']
Names Found: ['裴漠衍', '余清溪']
These arent names: ['老太太', '奶奶']
Names Found: ['墨寻', '唐知落', '穆九', '墨老太太']
These arent names: ['姐姐', '妹妹', '老板', '助理', '朋友', '同事']
Names Found: ['唐霄', '简瑶', '乔妹', '傅盛年']
These arent names: ['妻子', '丈夫', '姐姐', '妹妹', '爷爷', '奶奶', '爸爸', '妈妈']
Names Found: ['余清溪', '张晴', '余雯雯', '裴漠衍', '凌慕', '余家']
These arent names: []
Names Found: ['封薄言', '叶星语', '叶世华', '封华']
These arent names: ['妹妹', '姐姐', '朋友']
Names Found: ['盛年', '简瑶', '乔妹']
These arent names: ['中年妇女']
Names Found: ['冉香玉', '冉雨清', '张友华', '李桂花']
These arent names: ['宝宝']
Names Found: ['江政和', '冉雨清', '小宝']
These arent names: ['无']
Names Found: ['余清溪', '裴漠衍']
These arent names: ['无']
Names Found: ['温落', '霍廷翊', '温语']
These arent names: []
Names Found: ['霍廷翊', '刘晴婉', '温语', '温落', '孟星然']
These arent names: ['无']
Names Found: ['裴漠衍', '林逸', '余清溪']
These arent names: ['无']
Names Found: ['余清溪', '许诺']
These arent names: []
Names Fo

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wuzihao\AppData\Local\Temp\jieba.cache


These arent names: ['同学', '朋友', '同事', '伙伴']
Names Found: ['唐知落', '顾笙笙', '梁远哲', '唐知羡', '墨寻']


Loading model cost 0.364 seconds.
Prefix dict has been built successfully.


These arent names: ['女性', '男性', '名字']
Names Found: ['冉雨清', '冉香玉', '江政和', '李桂花']
Split: 8
These arent names: ['陆绾', '陆婷婷', '厉溟庭', '乔琨', '景林', '张峰']
Names Found: []
These arent names: ['导演', '女人', '男人', '助理', '老板', '总裁']
Names Found: ['陆绾', '景林', '红裙女人', '凯文', '陆婷婷', '张导', '乔琨', '厉溟庭']
These arent names: ['同学', '同事', '朋友', '伙伴', '学生', '老师', '领导', '下属', '前辈', '后辈', '室友', '同伴', '搭档', '队友']
Names Found: ['叶星语', '封薄言', '许牧', '苏颜颜', '谢青岑', '叶世华']
These arent names: ['丈夫', '妻子', '总裁', '警察', '宝宝', '爸爸', '妈妈', '爷爷', '奶奶', '姐姐', '妹妹', '哥哥', '弟弟']
Names Found: ['封墨霆', '南笙', '池城', '楚瑶', '龙一']
These arent names: ['丈夫', '妻子', '父亲', '母亲', '哥哥', '姐姐', '弟弟', '妹妹', '爷爷', '奶奶', '叔叔', '阿姨', '老板', '员工', '老师', '学生', '医生', '护士', '警察', '律师', '工程师', '程序员', '会计', '厨师', '服务员', '司机', '保安', '清洁工', '农民', '工人', '商人', '总裁', '秘书', '助理', '宝宝', '孩子', '成人', '老人']
Names Found: ['冉雨清', '李桂花', '江政和', '冉建军', '冉香玉']
Split: 7
These arent names: ['丈夫', '妻子', '哥哥', '姐姐', '弟弟', '妹妹', '爸爸', '妈妈', '爷爷', '奶奶', '老板', '总裁', '秘书', '助理',

In [4]:
train_documents = [x[1] for x in train_seg_lists]

In [5]:
from gensim import corpora

dictionary = corpora.Dictionary(train_documents)
corpus = [dictionary.doc2bow(x) for x in train_documents]

print(corpus[:16])

[[(0, 1), (1, 2), (2, 2), (3, 1), (4, 1), (5, 5), (6, 1), (7, 2), (8, 2), (9, 2), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 5), (22, 1), (23, 1), (24, 2), (25, 2), (26, 8), (27, 1), (28, 1), (29, 1), (30, 1), (31, 5), (32, 1), (33, 1), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 3), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 3), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 13), (59, 2), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 5), (81, 1), (82, 1), (83, 2), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 2), (106, 2), (107, 1), (108, 1), (109, 1), (110, 1

In [ ]:
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

num_topics = 16

for i in range(2, num_topics):
    lda_model = LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=i,
        passes=64,
        random_state=32
    )
    
    cm = CoherenceModel(model=lda_model, texts=train_documents, coherence='u_mass')
    coherence = cm.get_coherence()
    print(f"Coherence={coherence}", f"Topics={num_topics}")
    pyLDAvis.display(gensimvis.prepare(lda_model, corpus, dictionary))

In [7]:
# save model
lda_model.save('model/lda/lda_model')